# Herron Topic 3 - Practice - Sec 04

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import statsmodels.api as sm
import yfinance as yf

In [2]:
%precision 4
pd.options.display.float_format = '{:.4f}'.format
# %config InlineBackend.figure_format = 'retina'

## Announcements

1. Projects:
    1. Project 1:
        1. I am still grading
        2. I plan to finish grading by Friday
    2. Project 2:
        1. Due on Friday, 3/28, at 11:59 PM
        2. We will use class time next week for group work
        3. Ask your questions here: <https://northeastern.instructure.com/courses/207607/discussion_topics/2738173>
2. Assessments:
    1. ***Both assessments are in class on Tuesday, 4/15***
    2. Programming assessment: Should take 15 minutes and will be based on the questions here:
    3. MSFQ assessment (only for MSFQ students): Should take 45 minutes and based on the 5 required MSFQ courses (standby for a Canvas announcement with more details after I discuss it with the MSFQ program director)

## Five-Minute Review

Quantitative value investing is a systematic investment strategy that combines the principles of value investing with data-driven, quantitative techniques.
It involves identifying undervalued securities---those trading below their intrinsic value---using predefined, measurable criteria rather than subjective judgment.
Common criteria are price/earnings (P/E) and price/book (P/B) ratios.
Here, we will use the book-to-market equity ratio (B/M), which is common in the academic literature and quantitative investing.

The key concepts in this topic are creating metrics, forming portfolios, and backtesting the strategy.

##  Practice

### Re-implement the value strategy from the lecture notebook

#### Read the data

This code reads stock market data from a file called `crsp.csv` and prepares it for analysis.
It loads the data into a data frame, makes sure the dates are in the right format, and marks certain letters (`A`, `B`, and `C`) as missing values.
Then, it filters the data to include only records from 1965 or later, sorts it by company and date, and adds two new columns: one for market equity (calculated as the number of shares times the stock price, converted to millions), and another to adjust all dates to the end of each month.

In [3]:
crsp = (
    pd.read_csv(
        filepath_or_buffer='crsp.csv',  # Read data from crsp.csv file
        parse_dates=['date'],           # Convert 'date' column to datetime format
        na_values=['A', 'B', 'C']       # Treat 'A', 'B', 'C' as missing values (NaN)
    )
    .query('date >= 1965')              # Filter to keep only data from 1965 onward
    .sort_values(['PERMNO', 'date'])    # Sort by company identifier (PERMNO) and date
    .assign(
        ME=lambda x: x['SHROUT'] * x['PRC'].abs() / 1_000,  # Calculate market equity (ME) as shares outstanding * absolute price / 1000
        date=lambda x: x['date'] + pd.offsets.MonthEnd(0)   # Adjust dates to the last day of each month
    )
)

In [4]:
crsp.head()

,PERMNO,date,SHRCD,PRC,RET,SHROUT,ME
0,10000,1986-01-31,10,-4.3750,NaN,3680.0000,16.1000
1,10000,1986-02-28,10,-3.2500,-0.2571,3680.0000,11.9600
2,10000,1986-03-31,10,-4.4375,0.3654,3680.0000,16.3300
3,10000,1986-04-30,10,-4.0000,-0.0986,3793.0000,15.1720
4,10000,1986-05-31,10,-3.1094,-0.2227,3793.0000,11.7939


In [5]:
compustat = (
    pd.read_csv(
        filepath_or_buffer='compustat.csv',  # Read data from compustat.csv file
        parse_dates=['datadate']              # Convert 'datadate' column to datetime format
    )
    .query('datadate >= 1965')                # Filter to keep only data from 1965 onward
    .sort_values(['LPERMNO', 'fyear', 'datadate'])  # Sort by company identifier (LPERMNO), fiscal year, and date
    .drop_duplicates(subset=['LPERMNO', 'fyear'], keep='last')  # Keep only the latest record per company and fiscal year
)

In [6]:
compustat.head()

,GVKEY,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,ceq,ib,costat
165681,13007,10000,1986-10-31,1986.0000,INDL,C,D,STD,USD,0.4180,-0.7300,I
165586,12994,10001,1986-06-30,1986.0000,INDL,C,D,STD,USD,5.4320,0.6690,I
165587,12994,10001,1987-06-30,1987.0000,INDL,C,D,STD,USD,5.3690,0.3120,I
165588,12994,10001,1988-06-30,1988.0000,INDL,C,D,STD,USD,5.5120,0.5420,I
165589,12994,10001,1989-06-30,1989.0000,INDL,C,D,STD,USD,6.3210,1.2080,I


#### Create the interim data frames

This code finds the market value of equity (`ME`) as of December each year from the `crsp` data frame (to match with book value of equity `ceq` from the `compustat` data frame below).
It filters out any rows where market equity is zero or negative, then sorts the data by company and date.
Next, it groups the data by company and year, focusing on December values.
Finally, it selects the last market equity value for each company in each year, giving us the December `ME`.

In [7]:
mve = (
    crsp
    .query('ME > 0')
    .sort_values(by=['PERMNO', 'date'])
    .groupby(by=['PERMNO', pd.Grouper(key='date', freq='YE-DEC')])
    [['ME']]
    .last()
)

In [8]:
mve.head()

ME
PERMNO date             
10000  1986-12-31 1.9816
       1987-12-31 0.8516
10001  1986-12-31 6.9370
       1987-12-31 5.8280
       1988-12-31 6.3623

This code finds the book value of equity (`ceq`) as of December each year from the `compustat` data frame.
It filters out any rows where common equity is zero or negative, then sorts the data by company and date.
Next, it groups the data by company and year, focusing on December values.
Finally, it selects the last common equity value for each company in each year, giving us the December `ceq`.

In [9]:
bve = (
    compustat
    .query('ceq > 0')
    .sort_values(by=['LPERMNO', 'datadate'])
    .groupby(by=['LPERMNO', pd.Grouper(key='datadate', freq='YE-DEC')])
    [['ceq']]
    .last()
)

In [10]:
bve.head()

ceq
LPERMNO datadate         
10000   1986-12-31 0.4180
10001   1986-12-31 5.4320
        1987-12-31 5.3690
        1988-12-31 5.5120
        1989-12-31 6.3210

#### Combine the data frames and form portfolios

This code shows a simple example of how to form portfolios using the `pd.qcut()` function.
It takes a sequence of numbers from 0 to 9 and splits them into 2 equal groups (quantiles), assigning them to either a low or high category.
The `labels=False` argument gives us numeric group identifiers (0 or 1), and adding 1 shifts these to 1 or 2, which can represent portfolio numbers.

In [11]:
1 + pd.qcut(x=np.arange(10), q=2, labels=False)

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

::: {.callout-note}

In class, in the following cell, I used multiple cursors to simultaneously type two `.rename_axis(index=['PERMNO', 'Date'])` methods.
If you hold down the `CTRL` key, every mouse click will generate a new cursor.
Each cursor will do the same action (e.g., typing, highlighting, and deleting).
More [here](https://discourse.jupyter.org/t/most-useful-keyboard-shortcuts-for-notebook-lab/18113).

:::

::: {.callout-note}
Start here on Friday!
:::

In [12]:
(
    mve
    .rename_axis(index=['PERMNO', 'Date'])
    .join(
        other=bve.rename_axis(index=['PERMNO', 'Date']),
        how='inner'
    )
    .assign(BM=lambda x: x['ceq'] / x['ME'])
    .reset_index()
    
)

,PERMNO,Date,ME,ceq,BM
0,10000,1986-12-31,1.9816,0.4180,0.2109
1,10001,1986-12-31,6.9370,5.4320,0.7830
2,10001,1987-12-31,5.8280,5.3690,0.9212
3,10001,1988-12-31,6.3623,5.5120,0.8664
4,10001,1989-12-31,10.3477,6.3210,0.6109
...,...,...,...,...,...
246063,93436,2020-12-31,677340.1530,22225.0000,0.0328
246064,93436,2021-12-31,1092190.6152,30189.0000,0.0276
246065,93436,2022-12-31,389741.5200,44704.0000,0.1147
246066,93436,2023-12-31,791408.8000,62634.0000,0.0791


#### Backtest the strategy

### Re-implement the value strategy from the lecture notebook *with value-weighted portfolios*

### Estimate the $\alpha$s of the equal-weighted and value-weighted portfolios

### Implement a momentum strategy

Form deciles on the 11-month returns from months $t-12$ to month $t-1$.